In [1]:
import pandas as pd
import numpy as np
import requests
import bs4
from bs4 import BeautifulSoup

In [2]:
referencesList = []
externalLinksList = []
categoryList = []

In [3]:
df = pd.read_excel('churchesLinks.xlsx')
mylist = df['Link'].tolist()

In [4]:
def getReferences(html):
    stri = ""
    maxSiz = 7
    refDiv = html.find_all(class_="references")
    if len(refDiv) > 1:
        refDiv = refDiv[1]
    elif len(refDiv) == 1:
        refDiv = refDiv[0]
    if len(refDiv) != 0:
        allRefTexts = refDiv.find_all(class_="reference-text")
        for i in allRefTexts:
            if maxSiz == 0:
                break
            link = i.find('a',href=True)
            if link != None:
                link = link['href']
                link = ("https://en.wikipedia.org/" + link) if link.find('http') == -1  else link
                stri += (link + " ")
                maxSiz -= 1
    else:
        stri = "nan"
    if stri == "":
        stri = 'nan'
    return stri

In [5]:
def getExternalLinks(html):
    siz = 3
    externalLinkTag = html.find(attrs={'id':'External_links'})
    if externalLinkTag != None:
        externalLinkTag = externalLinkTag.parent
        externalLinkTag = externalLinkTag.next_sibling
        while str(externalLinkTag) == "\n" and externalLinkTag != None:
            externalLinkTag = externalLinkTag.next_sibling
        while siz > 0:
            link = externalLinkTag.find_all('a',href=True)
#             print(externalLinkTag)
            if len(link) != 0:
#                 if link.has_attrs("href"):
                for i in link:
                    if isinstance(i,bs4.element.Tag):
                        if i != None:
                                i = i['href']
                                if i.find('wiki') == -1:
                                    print(i)
                                    siz -= 1
            externalLinkTag = externalLinkTag.next_sibling
            while str(externalLinkTag) == "\n":
                externalLinkTag = externalLinkTag.next_sibling
            if externalLinkTag == None:
                break
    else:
        print("nan")
        

In [6]:
def getExternalLinksBYUl(html):
    stri = ""
    externalLinkTag = html.find(attrs={'id':'External_links'})
    if externalLinkTag != None:
        externalLinkTag = externalLinkTag.parent
        externalLinkTag = externalLinkTag.next_sibling
        while str(externalLinkTag) == "\n":
            externalLinkTag = externalLinkTag.next_sibling
        if externalLinkTag != None:
            for i in range(5):
                if externalLinkTag.name == 'ul':
                    break;
                externalLinkTag = externalLinkTag.next_sibling
                while str(externalLinkTag) == "\n":
                    externalLinkTag = externalLinkTag.next_sibling
                if externalLinkTag == None:
                    break
        if externalLinkTag != None and externalLinkTag.name == 'ul':
            link = externalLinkTag.find_all('a',href=True)
            for i in link:
                if i['href'].find('wiki') == -1:
                    stri += (i['href'] + " ")
        else:
            stri = 'nan'
    else:
        stri = 'nan'
    if stri == "":
        stri  = "nan"
    return stri

In [7]:
def getCategory(html):
    stri = ""
    catDiv = html.find(attrs={'id':'catlinks'})
    if catDiv != None:
        thatUl = catDiv.find('ul')
        if thatUl != None:
            allLinks = thatUl.find_all('a',href=True)
            for i in allLinks:
                stri += ('https://en.wikipedia.org/' + i['href'] + ' ')
        else:
            stri = "nan"
    else:
        stri = 'nan'
    if stri == "":
        stri = 'nan'
    return stri

In [ ]:
for i in range(len(mylist)):
    url = mylist[i]
#     url = "https://en.wikipedia.org/wiki/Id_Kah_Mosque"
    data = requests.get(url)
    html = BeautifulSoup(data.text, 'html.parser')
    referencesList.append(getReferences(html))
    externalLinksList.append(getExternalLinksBYUl(html))
    categoryList.append(getCategory(html))
#     print(getReferences(html))
#     print(getExternalLinksBYUl(html))
#     print(getCategory(html))
    print(i,"done")

In [ ]:
print(len(referencesList))
print(len(externalLinksList))
print(len(categoryList))

In [11]:
liste = pd.DataFrame({ "references": referencesList,"externalLinks": externalLinksList,"category":categoryList})
liste.to_excel("churcheRemaingRECscraped.xlsx")